## 0. Install Dependecies

In [1]:
pip install torch

**Set device to GPU if is available otherwise set device as cpu**

In [2]:
import torch
# Check if GPU is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


**Import libraries**

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.base import BaseEstimator, RegressorMixin

##1. Dataset##

- The Datatset used in this project was generated using the Mujoco simulator with three different configurations:
- 2D (2 joints)
- 2D (3 joints)
- 3D (5 joints)

The format of the data is in CSV format, including information about Joint angles, fingertip position, and orientation.


1.1. Visualise data from the simulator

In [3]:
!head -5 logfiler2.csv

j0;j1;cos(j0);cos(j1);sin(j0);sin(j1);ft_x;ft_y;ft_qw;ft_qz
 0.055; -0.012;  0.998;  1.000;  0.055; -0.012;  0.210;  0.010;  1.000;  0.021
 0.076; -0.017;  0.997;  1.000;  0.076; -0.017;  0.210;  0.014;  1.000;  0.030
 0.148; -0.011;  0.989;  1.000;  0.147; -0.011;  0.208;  0.030;  0.998;  0.068
 0.214;  0.048;  0.977;  0.999;  0.212;  0.048;  0.204;  0.050;  0.991;  0.131


In [ ]:
!head -5 logfiler3.csv

head: cannot open 'logfiler3.csv' for reading: No such file or directory


In [ ]:
!head -5 logfiler5.csv

head: cannot open 'logfiler5.csv' for reading: No such file or directory


1.2. Preprocess the data


2R Robot

In [8]:
# Load dataset
data = pd.read_csv('logfiler2.csv', delimiter=';')

# Preprocessing: Extract inputs (joint angles and their trigonometric functions) and outputs (fingertip positions and quaternions)
X = data[['j0', 'j1', 'cos(j0)', 'cos(j1)', 'sin(j0)', 'sin(j1)']].values
y = data[['ft_x' ,'ft_y' ,'ft_qw','ft_qz']].values

# Normalize input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

- Split the data into training and testing sets

In [9]:
# Split data into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

##2. Train Forward Kinematics Models##



### 2.1. Robot 2R

- Define the architecture of the model (Feedforward Neural Network) to learn forward kinematics.




In [28]:
class ForwardKinematicsModel(nn.Module):
    def __init__(self, hidden_size, dropout_rate=0.3):
        super(ForwardKinematicsModel, self).__init__()
        # Define a feedforward network with configurable hidden_size and dropout
        self.fc1 = nn.Linear(6, hidden_size)  # Input layer
        self.dropout1 = nn.Dropout(p=dropout_rate)    # Dropout after first hidden layer
        self.fc2 = nn.Linear(hidden_size, hidden_size)  # Second hidden layer
        self.dropout2 = nn.Dropout(p=dropout_rate)    # Dropout after second hidden layer
        self.fc3 = nn.Linear(hidden_size, 4)  # Output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)  # Apply dropout after first hidden layer
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)  # Apply dropout after second hidden layer
        x = self.fc3(x)
        return x

- Hyperparameter Search

Using Grid Search

In [36]:
# PyTorch Regressor Wrapper
class PyTorchRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, hidden_size=hidden_size  , lr=learning_rate, epochs=num_epochs):
        self.hidden_size = hidden_size
        self.lr = lr
        self.epochs = epochs
        self.model = ForwardKinematicsModel(hidden_size=self.hidden_size)
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        self.criterion = nn.MSELoss()

    def fit(self, X, y):
        X_train = torch.tensor(X, dtype=torch.float32)
        y_train = torch.tensor(y, dtype=torch.float32)
        for epoch in range(self.epochs):
            self.model.train()
            self.optimizer.zero_grad()
            output = self.model(X_train)
            loss = self.criterion(output, y_train)
            loss.backward()
            self.optimizer.step()
        return self

    def predict(self, X):
        X_test = torch.tensor(X, dtype=torch.float32)
        self.model.eval()
        with torch.no_grad():
            predictions = self.model(X_test)
        return predictions.numpy()

# Load your dataset
df = pd.read_csv('logfiler2.csv', sep=';')
X = df[['j0', 'j1', 'cos(j0)', 'cos(j1)', 'sin(j0)', 'sin(j1)']].values
y = df[['ft_x' ,'ft_y' ,'ft_qw','ft_qz']].values

# Split dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Set up hyperparameters to search
param_grid = {
    'hidden_size': [32, 64, 128],  # Try different numbers of hidden units
    'lr': [0.001, 0.01],           # Try different learning rates
    'epochs': [100, 200],          # Try different numbers of epochs
}

# Instantiate the model wrapper
pytorch_model = PyTorchRegressor()

# Use GridSearchCV
grid_search = GridSearchCV(estimator=pytorch_model, param_grid=param_grid, cv=3, verbose=2, n_jobs=1)
grid_search.fit(X_train, y_train)

# Get the best two combinations of hyperparameters
results = pd.DataFrame(grid_search.cv_results_)
results = results.sort_values(by='mean_test_score', ascending=False)

# Print the best and second-best hyperparameters
print("Best combination of hyperparameters:")
print(results.iloc[0][['param_hidden_size', 'param_lr', 'param_epochs', 'mean_test_score']])
print("\nSecond-best combination of hyperparameters:")
print(results.iloc[1][['param_hidden_size', 'param_lr', 'param_epochs', 'mean_test_score']])

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
test_loss = mean_squared_error(y_test, y_pred)
print(f"\nTest MSE of the best model: {test_loss:.4f}")

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] END ...............epochs=100, hidden_size=32, lr=0.001; total time=   5.1s
[CV] END ...............epochs=100, hidden_size=32, lr=0.001; total time=   4.8s
[CV] END ...............epochs=100, hidden_size=32, lr=0.001; total time=   4.6s
[CV] END ................epochs=100, hidden_size=32, lr=0.01; total time=   5.8s
[CV] END ................epochs=100, hidden_size=32, lr=0.01; total time=   4.2s
[CV] END ................epochs=100, hidden_size=32, lr=0.01; total time=   4.7s
[CV] END ...............epochs=100, hidden_size=64, lr=0.001; total time=   5.1s
[CV] END ...............epochs=100, hidden_size=64, lr=0.001; total time=   4.4s
[CV] END ...............epochs=100, hidden_size=64, lr=0.001; total time=   5.6s
[CV] END ................epochs=100, hidden_size=64, lr=0.01; total time=   4.9s
[CV] END ................epochs=100, hidden_size=64, lr=0.01; total time=   5.0s
[CV] END ................epochs=100, hidden_size

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best combination of hyperparameters:
param_hidden_size         128
param_lr                0.001
param_epochs              200
mean_test_score      0.783651
Name: 10, dtype: object

Second-best combination of hyperparameters:
param_hidden_size          32
param_lr                0.001
param_epochs              200
mean_test_score      0.775462
Name: 6, dtype: object

Test MSE of the best model: 0.0146


## Train the model with the best 2 parameters found by grid search

- Train the models on joint angle inputs to predict fingertip positions.

In [38]:
# Hyperparameters
hidden_size = 128
learning_rate = 0.001
num_epochs = 200

- Define the loss function and optimizer

In [39]:
model = ForwardKinematicsModel(hidden_size=hidden_size)

In [40]:
# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [41]:
# Convert to PyTorch tensors only if necessary
X_train = X_train.clone().detach().float() if isinstance(X_train, torch.Tensor) else torch.tensor(X_train, dtype=torch.float32)
y_train = y_train.clone().detach().float() if isinstance(y_train, torch.Tensor) else torch.tensor(y_train, dtype=torch.float32)
X_val = X_val.clone().detach().float() if isinstance(X_val, torch.Tensor) else torch.tensor(X_val, dtype=torch.float32)
y_val = y_val.clone().detach().float() if isinstance(y_val, torch.Tensor) else torch.tensor(y_val, dtype=torch.float32)
X_test = X_test.clone().detach().float() if isinstance(X_test, torch.Tensor) else torch.tensor(X_test, dtype=torch.float32)
y_test = y_test.clone().detach().float() if isinstance(y_test, torch.Tensor) else torch.tensor(y_test, dtype=torch.float32)

In [42]:
# Initialize variables for early stopping
best_val_loss = float('inf')
patience = 4
no_improvement_epochs = 0
best_model_path = "best_model.pth"  # Path to save the best model

# Training loop with early stopping
for epoch in range(num_epochs):
    model.train()

    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Evaluate on validation set
    if (epoch + 1) % 10 == 0:
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs, y_val)

            # Calculate additional metrics
            mae = mean_absolute_error(y_val.numpy(), val_outputs.numpy())
            r2 = r2_score(y_val.numpy(), val_outputs.numpy())

            print(f"Epoch [{epoch+1}/{num_epochs}], "
                  f"Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}, "
                  f"MAE: {mae:.4f}, R^2: {r2:.4f}")

            # Early stopping check
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                no_improvement_epochs = 0
                # Save the best model
                torch.save(model.state_dict(), best_model_path)
                print(f"Model saved at epoch {epoch+1} with Val Loss: {val_loss:.4f}")
            else:
                no_improvement_epochs += 1

            if no_improvement_epochs >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. Best Val Loss: {best_val_loss:.4f}")

        model.train()  # Switch back to training mode

Epoch [10/200], Train Loss: 0.1243, Val Loss: 0.1280, MAE: 0.2530, R^2: 0.2904
Model saved at epoch 10 with Val Loss: 0.1280
Epoch [20/200], Train Loss: 0.0694, Val Loss: 0.0811, MAE: 0.2091, R^2: 0.3298
Model saved at epoch 20 with Val Loss: 0.0811
Epoch [30/200], Train Loss: 0.0477, Val Loss: 0.0713, MAE: 0.1911, R^2: 0.4968
Model saved at epoch 30 with Val Loss: 0.0713
Epoch [40/200], Train Loss: 0.0344, Val Loss: 0.0691, MAE: 0.1867, R^2: 0.5137
Model saved at epoch 40 with Val Loss: 0.0691
Epoch [50/200], Train Loss: 0.0290, Val Loss: 0.0674, MAE: 0.1790, R^2: 0.6052
Model saved at epoch 50 with Val Loss: 0.0674
Epoch [60/200], Train Loss: 0.0259, Val Loss: 0.0616, MAE: 0.1686, R^2: 0.6574
Model saved at epoch 60 with Val Loss: 0.0616
Epoch [70/200], Train Loss: 0.0234, Val Loss: 0.0631, MAE: 0.1706, R^2: 0.6372
Epoch [80/200], Train Loss: 0.0218, Val Loss: 0.0654, MAE: 0.1745, R^2: 0.6237
Epoch [90/200], Train Loss: 0.0205, Val Loss: 0.0655, MAE: 0.1748, R^2: 0.6323
Epoch [100/20

In [ ]:
# Test the model
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test)
    mae_test = mean_absolute_error(y_test.numpy(), test_outputs.numpy())
    r2_test = r2_score(y_test.numpy(), test_outputs.numpy())

    print(f"Test Loss: {test_loss.item():.4f}")
    print(f"Test MAE: {mae_test:.4f}")
    print(f"Test R^2: {r2_test:.4f}")


Test Loss: 0.0041
Test MAE: 0.0505
Test R^2: 0.5014


##3. Compare Jacobians##



3.1. Compute the Jacobian matrix for the learned forward kinematics using automatic differentiation.



In [ ]:
# Load your data (replace this with your actual dataset loading method)
df = pd.read_csv('logfiler2.csv', sep=';')
X = df[['j0', 'j1']].values  # Joint angles
y = df[['ft_x', 'ft_y']].values  # Fingertip positions

# Convert to torch tensor
X_test = torch.tensor(X, dtype=torch.float32)

In [49]:
# Assuming X_test is a tensor and you want to extract the first sample
X_test_sample = X_test[0].clone().detach().unsqueeze(0).float()  # Add batch dimension and detach
print(X_test_sample)

tensor([[-3.0970, -2.6260, -0.9990, -0.8700, -0.0450, -0.4930]])


In [ ]:
# Load the best model after training
model.load_state_dict(torch.load(best_model_path))
print("Best model loaded for further evaluation or inference.")

In [58]:
def compute_2x2_jacobian(model, inputs):
    # Ensure input gradients are enabled
    if not inputs.requires_grad:
        inputs.requires_grad = True

    # Forward pass
    outputs = model(inputs)

    # Select outputs of interest
    ft_x, ft_y = outputs[0], outputs[1]
    print("ft_x:", ft_x)
    print("ft_y:", ft_y)

    # Initialize the 2x2 Jacobian matrix
    jacobian = torch.zeros(2, 2)

    # Compute gradients for ft_x
    grad_ft_x = torch.autograd.grad(ft_x, inputs, retain_graph=True, create_graph=True)[0]
    jacobian[0, 0] = grad_ft_x[0]  # ∂ft_x/∂j0
    jacobian[0, 1] = grad_ft_x[1]  # ∂ft_x/∂j1

    # Compute gradients for ft_y
    grad_ft_y = torch.autograd.grad(ft_y, inputs, retain_graph=True, create_graph=True)[0]
    jacobian[1, 0] = grad_ft_y[0]  # ∂ft_y/∂j0
    jacobian[1, 1] = grad_ft_y[1]  # ∂ft_y/∂j1

    return jacobian

3.2. Compare the computed Jacobian with the analytical Jacobian for the 2-joint robot.

In [53]:
import numpy as np

# Analytical Jacobian for a 2-DOF robot
def analytical_jacobian_from_test(data):
    # Extract joint angles from data
    j0 = data[0]  # j0
    j1 = data[1]  # j1

    l1 = 1.0  # Link 1 length
    l2 = 1.0  # Link 2 length

    # Calculate the Jacobian elements using the analytical formula
    J = np.array([
        [-l1 * np.sin(j0) - l2 * np.sin(j0 + j1), -l2 * np.sin(j0 + j1)],
        [l1 * np.cos(j0) + l2 * np.cos(j0 + j1), l2 * np.cos(j0 + j1)]
    ])
    return J

In [62]:
import numpy as np
import torch

#Numerical Jacobian (calculated by FK_Jacobian)
X_test_sample = torch.tensor([-3.0970, -2.6260, -0.9990, -0.8700, -0.0450, -0.4930], requires_grad=True)
jacobian_numerical = compute_2x2_jacobian(model, X_test_sample)

# Analytical Jacobian (calculated using analytical formula)X_test_sample = [-3.0970, -2.6260, -0.9990, -0.8700, -0.0450, -0.4930] # Extract joint angles for comparison
X_test_sample = X_test_sample.detach().numpy()
J_analytical = analytical_jacobian_from_test(X_test_sample)

# Print both Jacobians
print("Numerical Jacobian (PyTorch):\n", jacobian_numerical)
print("Analytical Jacobian (2-DOF Robot):\n", J_analytical)

# Compare both Jacobians by calculating the difference
jacobian_difference = (jacobian_numerical.detach().numpy() - J_analytical)
print("Difference between Numerical and Analytical Jacobians:\n", jacobian_difference)

Numerical Jacobian (PyTorch):
 tensor([[ 0.0068, -0.0159],
        [ 0.0118, -0.0285]], grad_fn=<CopySlices>)
Analytical Jacobian (2-DOF Robot):
 [[-0.48676559 -0.53134358]
 [-0.15184945  0.84715647]]
Difference between Numerical and Analytical Jacobians:
 [[ 0.49353678  0.51546606]
 [ 0.16368002 -0.8756997 ]]
